In [4]:
import shutil, os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import matplotlib.pyplot as plt
import numpy as np
import detectron2
from detectron2.data import DatasetCatalog,MetadataCatalog
from plain_train_net import grab_dataset
import detectron2.utils.comm as comm
from detectron2.engine import launch


reprocess_results=True
# dataset_name = "reserves_fold1"
dataset_name = "Test Set"
dpi=120

In [6]:
#predict
from detectron2.config import get_cfg
cfg = get_cfg()
cfg.merge_from_file('configs/working.yaml')
#really low iou for nms in order to separate out lesions
# cfg.merge_from_list(["MODEL.WEIGHTS", "output_valid_fold1/model_final.pth",
#                      "OUTPUT_DIR", "output_valid_"+ dataset_name + "/results"])
#print(cfg.dump())  # print formatted configs
print(cfg.MODEL.ROI_HEADS.dump())
print(cfg.MODEL.WEIGHTS)
print(cfg.OUTPUT_DIR)


BATCH_SIZE_PER_IMAGE: 512
IN_FEATURES:
- p2
- p3
- p4
- p5
IOU_LABELS:
- 0
- 1
IOU_THRESHOLDS:
- 0.5
NAME: StandardROIHeads
NMS_THRESH_TEST: 0.01
NUM_CLASSES: 1
POSITIVE_FRACTION: 0.25
PROPOSAL_APPEND_GT: true
SCORE_THRESH_TEST: 0.001

detectron2://ImageNetPretrained/FAIR/X-101-32x8d.pkl
./output


In [5]:
for name in [dataset_name]:
    try:
        DatasetCatalog.register(name, grab_dataset(name))
    except:
        print('Already registered.')
        #do nothing
    MetadataCatalog.get(name).thing_classes = ["rpd"]

# Prediction


In [ ]:

#%%script false --no-raise-error
#os.environ["NCCL_DEBUG"] = "INFO"
#from torch.nn.parallel import DistributedDataParallel
from detectron2.modeling import build_model
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
#from plain_train_net import EvaluateClass
from detectron2.evaluation import inference_on_dataset, COCOEvaluator

def predict_func(cfg,dataset_name):
    model = build_model(cfg)  # returns a torch.nn.Module
    
    # distributed = comm.get_world_size() > 1
    # if distributed:
    #     model = DistributedDataParallel(
    #         model, device_ids=[comm.get_local_rank()], broadcast_buffers=False
    #     )
    
    myloader = build_detection_test_loader(cfg,dataset_name) 
    myeval = COCOEvaluator(dataset_name,tasks={'bbox','segm'},output_dir ="output_"+ dataset_name) #produces _coco_format.json when initialized


    for mdl in ("fold1", "fold2", "fold3", "fold4","fold5"):
        #build model
        model_weights_path = "/data/amd-data/cera-rpd/detectron2-rpd/output_valid_"+ mdl +"/model_final.pth"
        DetectionCheckpointer(model).load(model_weights_path);  # load a file, usually from cfg.MODEL.WEIGHTS
        model.eval(); #set model in evaluation mode
        myeval.reset()
        output_dir = "output_"+ dataset_name + "/"+mdl
        myeval._output_dir = output_dir
        print("running inference on ", mdl)
        results_i = inference_on_dataset(model, myloader, myeval) #produces coco_instance_results.json when myeval.evaluate is called
    
    return  
predict_func(cfg,dataset_name)
print("done predict_func")
# launch(
#     predict_func,
#     6, # of gpus
#     num_machines=1,
#     machine_rank=0,
#     dist_url="auto",
#     args=(cfg,dataset_name,), #args to predict_func
# )

# Ensembling

In [3]:
#%%script false --no-raise-error
from Ensembler import Ensembler
ens = Ensembler('output_'+dataset_name,dataset_name,["fold1", "fold2", "fold3", "fold4","fold5"],.2)
ens.mean_score_nms()
ens.save_coco_instances()

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.25s)
creating index...
index created!
Successfully loaded output_Test Set/fold1/coco_instances_results.json into memory. 31392 instance detected.

Loading and preparing results...
DONE (t=0.38s)
creating index...
index created!
Successfully loaded output_Test Set/fold2/coco_instances_results.json into memory. 29391 instance detected.

Loading and preparing results...
DONE (t=0.33s)
creating index...
index created!
Successfully loaded output_Test Set/fold3/coco_instances_results.json into memory. 32281 instance detected.

Loading and preparing results...
DONE (t=0.33s)
creating index...
index created!
Successfully loaded output_Test Set/fold4/coco_instances_results.json into memory. 26330 instance detected.

Loading and preparing results...
DONE (t=0.18s)
creating index...
index created!
Successfully loaded output_Test Set/fold5/coco_instances_results.json into m

  2%|▏         | 268/13813 [00:02<02:30, 90.26it/s]


KeyboardInterrupt: 

# Evaluation

## Ensemble

In [7]:
from plain_train_net import EvaluateClass,CreatePlotsRPD
import json
#evaluate ensemble
myeval = EvaluateClass(
    dataset_name, "output_"+ dataset_name,iou_thresh = .2,prob_thresh=0.5,evalsuper=False)
myeval.evaluate()
%precision 3
print(myeval.summarize_scalars())
with open(os.path.join("output_"+ dataset_name,'scalar_dict.json'),"w") as outfile:
    json.dump(obj=myeval.summarize_scalars(),fp=outfile)
#num_images = myeval.num_images
#myeval.plot_PRcurve()
#plt.tight_layout()
#myeval.plot_recall_vs_prob()
#plt.tight_layout()

RPDplt = CreatePlotsRPD.initfromcoco(myeval.mycoco,myeval.prob_thresh)
#RPDplt.gt_dt_FP_FN_count()


Using previously cached COCO format annotations at 'output_Test Set/Test Set_coco_format.json'. You need to clear the cache file if your dataset has been modified.


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.51s)
creating index...
index created!
Calculated metrics for 13813 images
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.18s).
Accumulating evaluation results...
DONE (t=0.06s).
Using alternate calculation for fpr at instance score threshold of 0.5
{'dataset': 'Test Set', 'precision': -1.0, 'recall': 0.0, 'fpr': 0.14775935702037257, 'iou': 0.2, 'probability': 0.5}
Using alternate calculation for fpr at instance score threshold of 0.5


## Individual Models

In [ ]:
#evaluate individual models
import json
myeval.evalsuper=False
for mdl in ("fold1", "fold2", "fold3", "fold4","fold5"):
    myeval.reset()
    output_dir = "output_"+ dataset_name + "/"+mdl
    myeval._output_dir = output_dir
    myeval.evaluate()
    print(myeval.summarize_scalars())
    with open(os.path.join(output_dir,'scalar_dict.json'),"w") as outfile:
        json.dump(obj=myeval.summarize_scalars(),fp=outfile)


In [ ]:
import json, os
import pandas as pd
from table_styles import styles
mydicts=[]
for mdl in ['fold1','fold2','fold3','fold4','fold5']:
    output_dir = "output_"+ dataset_name + "/"+mdl
    with open(os.path.join(output_dir,'scalar_dict.json')) as f:
        mydicts.append(json.load(f))
with open(os.path.join("output_"+ dataset_name,'scalar_dict.json')) as f:
    mydicts.append(json.load(f))
dfr = pd.DataFrame(mydicts)
dfr = dfr.assign(fp = np.int32(dfr.fpr*num_images))
dfr = dfr.assign(f1 = 2*(dfr.precision*dfr.recall)/(dfr.precision + dfr.recall))
dfr = dfr.assign(model = ['1','2','3','4','5','ensemble'])
# dfr = dfr[['model','fpr','fp','probability','dataset']]
dfr = dfr[['model','dataset','precision','recall','f1','fpr','fp','iou','probability']]
pd.set_option('display.precision',3)
dfr.style.set_table_styles(styles).set_table_attributes('style="font-size: 17px"')

# Visualize Output

In [ ]:

from plain_train_net import OutputVis

dataset_name = 'Fold 1'
pred_file = "output_"+ dataset_name + "/coco_instances_results.json"
out_file = os.path.join("output_"+ dataset_name,'mean_score_nms_'+dataset_name+'_nonrpd.pdf')

# mdl = 'fold1'
# pred_file = os.path.join("output_"+ dataset_name,mdl,"coco_instances_results.json")
# out_file = os.path.join("output_"+ dataset_name,'mean_score_nms_'+mdl+'.pdf')

# vis = OutputVis(dataset_name,prob_thresh = 0.5,pred_mode='file',pred_file=pred_file,has_annotations=False)
# ImgIds = RPDplt.dfimg.index[RPDplt.dfimg.dt_instances>0] #np.abs(df.gt_xpxs-df.dt_xpxs).sort_values(ascending=False).index[0:50].values
# vis.output_to_pdf(ImgIds,out_file)

df = RPDplt.dfimg
vis = OutputVis(dataset_name,prob_thresh = 0.5,pred_mode='file',pred_file=pred_file,has_annotations=True)
ImgIds = RPDplt.dfimg.sample(50).index.values
#ImgIds = np.abs(df.gt_xpxs-df.dt_xpxs).sort_values(ascending=False).index[0:10].values
vis.output_to_pdf(ImgIds,out_file)


# Creating a TIF file from Binary Masks (No Overlay)

In [ ]:
from plain_train_net import OutputVis
from PIL import Image,ImageOps,ImageFilter,ImageSequence
from detectron2.utils.visualizer import Visualizer

dataset_name = 'Fold 1'
pred_file = "output_"+ dataset_name + "/coco_instances_results.json"

dfimg_dummy = RPDplt.dfimg.sort_index()
df_unique = dfimg_dummy.iloc[::49, :]
vis = OutputVis(dataset_name,prob_thresh = 0.5,pred_mode='file',pred_file=pred_file,has_annotations=True)

for scan in range(len(df_unique.index)):
    ImgIds = dfimg_dummy.head(49).index.values
    vis.output_masks_to_tiff(ImgIds, dfimg_dummy.loc[ImgIds[0],"ptid"], dfimg_dummy.loc[ImgIds[0], "eye"])
    dfimg_dummy = dfimg_dummy.iloc[49:]
    if dfimg_dummy.empty:
        #print('DataFrame is empty!')
        break

# Creating a TIF file from Binary Masks (With Overlay)

In [8]:
from plain_train_net import OutputVis
from PIL import Image,ImageOps,ImageFilter,ImageSequence
from detectron2.utils.visualizer import Visualizer

dataset_name = 'Test Set'
pred_file = "output_"+ dataset_name + "/coco_instances_results.json"

dfimg_dummy = RPDplt.dfimg.sort_index()
df_unique = dfimg_dummy.ptid.unique()
vis = OutputVis(dataset_name,prob_thresh = 0.5,pred_mode='file',pred_file=pred_file,has_annotations=True)
for scan in range(len(df_unique)):
    df_currentpt = dfimg_dummy.loc[dfimg_dummy['ptid'] == df_unique[scan]]
    df_pt_OD = df_currentpt.loc[df_currentpt['eye'] == 'OD']
    df_pt_OS = df_currentpt.loc[df_currentpt['eye'] == 'OS']
    df_pt_OD_ids = df_pt_OD.index.values
    df_pt_OS_ids = df_pt_OS.index.values
    #print(len(df_pt_OD.index))
    #print(len(df_pt_OS_ids))
    if (len(df_pt_OD.index) > 0):
        print("here!")
        vis.output_overlay_masks_to_tiff(df_pt_OD_ids, df_unique[scan], 'OD')
    if (len(df_pt_OS.index) > 0):
        print("now here!")
        vis.output_overlay_masks_to_tiff(df_pt_OS_ids, df_unique[scan], 'OS')

here!


KeyboardInterrupt: 

In [10]:
from plain_train_net import OutputVis
from PIL import Image,ImageOps,ImageFilter,ImageSequence
from detectron2.utils.visualizer import Visualizer

dataset_name = 'Test Set'
pred_file = "output_"+ dataset_name + "/coco_instances_results.json"

dfimg_dummy = RPDplt.dfimg.sort_index()
df_unique = dfimg_dummy.ptid.unique()
df_has_in = dfimg_dummy.loc[dfimg_dummy['dt_instances'] > 0 ]
print(df_has_in)

                       gt_instances  gt_pxs  gt_xpxs  dt_instances  dt_pxs  \
101002_OD_oct-000.png           0.0     0.0      0.0           0.0     0.0   
101002_OD_oct-001.png           0.0     0.0      0.0           0.0     0.0   
101002_OD_oct-002.png           0.0     0.0      0.0           0.0     0.0   
101002_OD_oct-003.png           0.0     0.0      0.0           0.0     0.0   
101002_OD_oct-004.png           0.0     0.0      0.0           0.0     0.0   
...                             ...     ...      ...           ...     ...   
501023_OS_oct-044.png           0.0     0.0      0.0           1.0   150.0   
501023_OS_oct-045.png           0.0     0.0      0.0           1.0    84.0   
501023_OS_oct-046.png           0.0     0.0      0.0           0.0     0.0   
501023_OS_oct-047.png           0.0     0.0      0.0           0.0     0.0   
501023_OS_oct-048.png           0.0     0.0      0.0           0.0     0.0   

                       dt_xpxs    ptid eye     scan  
101002_OD

# Creating a TIF file from Instance Masks (With Probability Labels)

In [ ]:
from plain_train_net import OutputVis
from PIL import Image,ImageOps,ImageFilter,ImageSequence
from detectron2.utils.visualizer import Visualizer

dataset_name = 'Fold 1'
pred_file = "output_"+ dataset_name + "/coco_instances_results.json"

dfimg_dummy = RPDplt.dfimg.sort_index()
df_unique = dfimg_dummy.iloc[::49, :]
vis = OutputVis(dataset_name,prob_thresh = 0.5,pred_mode='file',pred_file=pred_file,has_annotations=True)

for scan in range(len(df_unique.index)):
    ImgIds = dfimg_dummy.head(49).index.values
    vis.output_instances_masks_to_tiff(ImgIds, dfimg_dummy.loc[ImgIds[0],"ptid"], dfimg_dummy.loc[ImgIds[0], "eye"])
    dfimg_dummy = dfimg_dummy.iloc[49:]
    if dfimg_dummy.empty:
        #print('DataFrame is empty!')
        break

In [ ]:
from PIL import Image,ImageOps,ImageFilter,ImageSequence
#Opening tif file to see if it was successful
mfile = "extracted_test/101007_OD-bmasks.tif"
msk = Image.open(mfile)
for i,page in enumerate(ImageSequence.Iterator(msk)):
    page.save("msk_extracted/msk-{:03d}.png".format(i))     

In [ ]:
from PIL import Image,ImageOps,ImageFilter,ImageSequence
#Opening tif file to see if it was successful
mfile = "extracted_test_overlays/101002_OD-bmasks.tif"
msk = Image.open(mfile)
for i,page in enumerate(ImageSequence.Iterator(msk)):
    page.save("msk_extracted_overlays/msk-{:03d}.png".format(i))    

In [ ]:
from PIL import Image,ImageOps,ImageFilter,ImageSequence
#Opening tif file to see if it was successful
mfile = "instances_mask_overlays/101002_OD-ipmasks_overlay.tif"
msk = Image.open(mfile)
for i,page in enumerate(ImageSequence.Iterator(msk)):
    page.save("msk_instances_overlays/msk-{:03d}.png".format(i))    

In [ ]:
from table_styles import styles
dfpts = RPDplt.dfpts.sort_values(by=['dt_instances'],ascending=False)
html_str = dfpts.style.format('{:.0f}').set_table_styles(styles).render()
html_file = open(os.path.join('output_'+ dataset_name + '/dfpts_'+dataset_name+'.html'),'w')
html_file.write(html_str)
html_file.close()

In [ ]:
dfimg = RPDplt.dfimg.sort_index()
html_str = dfimg.style.set_table_styles(styles).render()
html_file = open(os.path.join('output_'+ dataset_name + '/dfimg_'+dataset_name+'.html'),'w')
html_file.write(html_str)
html_file.close()

In [ ]:
dfpts.sum()

In [ ]:
print(df_currentpt)
df_pt_OD = df_currentpt.loc[df_currentpt['eye'] == 'asdsadasd']
print(df_pt_OD)
if (len(df_pt_OD.index) == 0 ):
    print("empty! washoo!")